<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
    FIT-HCMUS, VNU-HCM 
    <br>
    Introduction To Data Science 
    <br>
    Final project 📌
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 02 - Preprocessing 📌
</div>

## **1. Set up enviroment**

In [1]:
# !pip install unidecode
# !pip install fuzzywuzzy
import pandas as pd
import numpy as np
import re
import warnings
from unidecode import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

/home/minhdeptrai/miniconda3/envs/min_ds-env/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## **2. Reading and cleaning data**

### **2.1. Fbref data**

#### **Raw data inspect**

In [2]:
raw_data_fbref = pd.read_csv("../../data/raw/raw_data_fbref.csv")
raw_data_fbref.head()

,Name,Position,Preferred Foot,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls.1,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches
0,William Saliba,"DF (CB, right)",Right,2018-2019,17.0,Saint-Étienne,fr FRA,1. Ligue 1,4th,16.0,13.0,"1,277",14.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.3,0.3,0.0,0.3,7.0,21.0,4.0,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Matches
1,William Saliba,"DF (CB, right)",Right,2019-2020,18.0,Saint-Étienne,fr FRA,1. Ligue 1,17th,12.0,11.0,992,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.1,0.3,20.0,33.0,7.0,0.00,0.0,0.00,0.00,0.00,0.02,0.01,0.03,0.02,0.03,Matches
2,William Saliba,"DF (CB, right)",Right,2020-2021,19.0,Nice,fr FRA,1. Ligue 1,9th,20.0,20.0,"1,800",20.0,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.8,0.8,0.0,0.9,36.0,106.0,3.0,0.05,0.0,0.05,0.05,0.05,0.04,0.00,0.04,0.04,0.04,Matches
3,William Saliba,"DF (CB, right)",Right,2020-2021,19.0,Arsenal,eng ENG,Jr. PL2 — Div. 1,10th,6.0,6.0,526,5.8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,Matches
4,William Saliba,"DF (CB, right)",Right,2021-2022,20.0,Marseille,fr FRA,1. Ligue 1,2nd,36.0,36.0,"3,240",36.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.7,1.7,0.4,2.1,50.0,220.0,14.0,0.00,0.0,0.00,0.00,0.00,0.05,0.01,0.06,0.05,0.06,Matches


**Checking data shape**

In [3]:
fbref_shape = raw_data_fbref.shape

In [4]:
# testing if data include enough 1000 items
print(f"Current shape: {fbref_shape}")

if fbref_shape[0] > 1000:
    print(f"Data is good!")
else:
    print(f"Raw data is absolutely small. Please recheck!")

Current shape: (3944, 39)
Data is good!


**Checking duplicated rows**

In [5]:
index = raw_data_fbref.index
detect_dup_series = index.duplicated(keep='first')
num_duplicated_rows = detect_dup_series.sum()

In [6]:
# checking containing duplicated rows
if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated rows.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate the data.!")

Raw data have no duplicated rows.!


**Column information**

In [7]:
raw_data_fbref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3944 entries, 0 to 3943
Data columns (total 39 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            3944 non-null   object 
 1   Position        3944 non-null   object 
 2   Preferred Foot  3944 non-null   object 
 3   Season          3930 non-null   object 
 4   Age             3930 non-null   float64
 5   Squad           3930 non-null   object 
 6   Country         3930 non-null   object 
 7   Comp            3930 non-null   object 
 8   LgRank          3930 non-null   object 
 9   MP              3930 non-null   float64
 10  Starts          3926 non-null   float64
 11  Min             3740 non-null   object 
 12  90s             3741 non-null   float64
 13  Gls             3739 non-null   float64
 14  Ast             3714 non-null   float64
 15  G+A             3714 non-null   float64
 16  G-PK            3738 non-null   float64
 17  PK              3738 non-null   f

It seems there might be an issue with displaying the column names correctly due to duplicates. We'll modify the column names to ensure they present suitable meanings.

In [8]:
new_cols_name = ['Name', 'Position', 'PreferredFoot', 'Season', 'Age', 'Squad', 'Country', 
                 'Comp', 'LgRank', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 
                 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
                 'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90', 'G+A-PK/90', 'xG/90', 
                 'xAG/90', 'xG+xAG/90', 'npxG/90', 'npxG+xAG/90', 'Matches']

column_name_mapping = dict(zip(raw_data_fbref.columns, new_cols_name))
raw_data_fbref.rename(columns=column_name_mapping, inplace=True)

raw_data_fbref.columns

Index(['Name', 'Position', 'PreferredFoot', 'Season', 'Age', 'Squad',
       'Country', 'Comp', 'LgRank', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls/90', 'Ast/90', 'G+A/90',
       'G-PK/90', 'G+A-PK/90', 'xG/90', 'xAG/90', 'xG+xAG/90', 'npxG/90',
       'npxG+xAG/90', 'Matches'],
      dtype='object')

**Field meanings**

| Field            | Description|
|------------------|---------|
| **Name**         | The name ò the player|
| **Season**       | Season|
| **Age**          | Age at season start|
| **Squad**        | The team the player is playing for|
| **Position**     | Position on the field|
| **PreferredFoot**| Foot that the player prefer using|
| **Country**      | Name of the country|
| **Comp**         | Competition|
| **LgRank**       | Squad finish in competition|
| **MP**           | Matches Played|
| **Starts**       | Games that player in starting lineup|
| **Min**          | Minutes played|
| **90s**          | 90s Played (Minutes played divided by 90)|
| **Gls**          | Goals scored or allowed|
| **Ast**          | Assists|
| **G+A**          | Goals + Assists|
| **G-PK**         | Non-Penalty Goals|
| **PK**           | Penalty Kicks Made|
| **PKatt**        | Penalty Kicks Attempted|
| **CrdY**         | Yellow Cards|
| **CrdR**         | Red Cards|
| **xG**           | Expected Goals (xG totals include penalty kicks, excluding penalty shootouts)|
| **npxG**         | Non-Penalty Expected Goals|
| **xAG**          | Expected Assisted Goals|
| **npxG+xAG**     | Non-Penalty Expected Goals plus Assisted Goals|
| **PrgC**         | Progressive Carries (Carries that move the ball towards the opponent's goal line)|
| **PrgP**         | Progressive Passes (Completed passes that move the ball towards the opponent's goal line)|
| **PrgR**         | Progressive Passes Received (Completed passes that move the ball towards the opponent's goal line)|
| **Gls/90**       | Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **Ast/90**       | Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **G+A/90**       | Goals and Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **G-PK/90**      | Non-Penalty Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **G+A-PK/90**    | Non-Penalty Goals plus Assists per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **xG/90**        | Expected Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **xAG/90**       | Expected Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **xG+xAG/90**    | Expected Goals plus Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **npxG/90**      | Non-Penalty Expected Goals per 90 minutes (Minimum 30 minutes played per squad game to qualify as a leader)|
| **npxG+xAG/90**  | Non-Penalty Expected Goals plus Assisted Goals per 90 minutes (Minimum 30 minutes played per squad game)|

**Removing meaningless rows**

- In certain football leagues, players may miss a season due to various reasons such as injuries, non-participation, suspensions, transfers, and more. This results in rows with incomplete information in the dataset. Therefore, we are contemplating the removal of these rows.

In [9]:
raw_data_fbref.dropna(subset=['Season'], inplace=True)
raw_data_fbref.shape

(3930, 39)

- Some players were registered but did not attempt any games due to some specific reasons (not suitable for tactic,...), rendering their data irrelevant. It is necessary to delete these rows.

In [10]:
raw_data_fbref = raw_data_fbref[raw_data_fbref['MP'] != 0]
raw_data_fbref.shape

(3740, 39)

- Goalkeepers are rarely injured, so we can drop them as collecting data in this group will not yield the demand proportion

In [11]:
raw_data_fbref = raw_data_fbref[raw_data_fbref['Position'] != ' GK']
raw_data_fbref.shape

(3399, 39)

**Remove unnecessary columns**

In [12]:
raw_data_fbref['Matches'].unique()

array(['Matches', nan], dtype=object)

Column `Matches` only has Matches value. In fact, on website this is a hyperlink to another match so dropping this column is necessary

In [13]:
raw_data_fbref = raw_data_fbref.drop(['Matches'],axis = 1)
raw_data_fbref.columns

Index(['Name', 'Position', 'PreferredFoot', 'Season', 'Age', 'Squad',
       'Country', 'Comp', 'LgRank', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast',
       'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG',
       'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls/90', 'Ast/90', 'G+A/90',
       'G-PK/90', 'G+A-PK/90', 'xG/90', 'xAG/90', 'xG+xAG/90', 'npxG/90',
       'npxG+xAG/90'],
      dtype='object')

**Reformating features and Adjusting data types**

- Overall, data type of most fields is correct except for minutes played `Min`, which should be integer instead of object. Here we're going to remove the comma in each value.

In [14]:
def format_min(min):
     if pd.notna(min):
        return int(min.replace(',', ''))
     else:
        return min
    
raw_data_fbref['Min'] = raw_data_fbref['Min'].apply(format_min)

- Additional players personal infos, and fundamental statistics such as `MP`, `Gls`, `Ast`, `G+A`, etc., need to be converted to integers. However, a challenge arises because these columns contain NaN values, which are represented as floating-point numbers, often indicating missing data. To facilitate the conversion of these columns to integers, the transformation from NaN to zero must be performed prior to converting the remaining values.

In [15]:
int_cols = ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 
            'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 
            'PrgC', 'PrgP', 'PrgR']

raw_data_fbref[int_cols] = raw_data_fbref[int_cols].fillna(0).astype(int)
raw_data_fbref.dtypes

Name              object
Position          object
PreferredFoot     object
Season            object
Age                int64
Squad             object
Country           object
Comp              object
LgRank            object
MP                 int64
Starts             int64
Min                int64
90s              float64
Gls                int64
Ast                int64
G+A                int64
G-PK               int64
PK                 int64
PKatt              int64
CrdY               int64
CrdR               int64
xG               float64
npxG             float64
xAG              float64
npxG+xAG         float64
PrgC               int64
PrgP               int64
PrgR               int64
Gls/90           float64
Ast/90           float64
G+A/90           float64
G-PK/90          float64
G+A-PK/90        float64
xG/90            float64
xAG/90           float64
xG+xAG/90        float64
npxG/90          float64
npxG+xAG/90      float64
dtype: object

- Extract the the teams' places in `LgRank` columns.

In [16]:
raw_data_fbref['LgRank'] = raw_data_fbref['LgRank'].str.extract('(\d+)').astype(int)
raw_data_fbref['LgRank'].unique()

array([ 4, 17,  9, 10,  2,  1, 11,  5, 13, 16,  6,  7, 14,  8,  3, 19, 12,
       15, 18, 20, 26, 28, 23, 22, 24, 21, 25])

- Remove redundant letters in values of `Country` column, which in fact is the national flags, causing by crawling process.

In [17]:
# Countries names format and their frequency
count = raw_data_fbref["Country"].value_counts()
count

Country
eng ENG    2372
fr FRA      169
de GER      143
es ESP      121
nl NED      110
it ITA      104
be BEL       78
pt POR       61
sct SCO      37
br BRA       34
ar ARG       29
dk DEN       24
ua UKR       14
at AUT       13
se SWE       12
cz CZE       11
us USA       10
ch SUI        9
no NOR        7
co COL        7
jp JPN        6
tr TUR        6
uy URU        5
pl POL        3
ec ECU        3
mx MEX        3
hr CRO        2
ru RUS        1
rs SRB        1
sa KSA        1
gr GRE        1
kr KOR        1
za RSA        1
Name: count, dtype: int64

In [18]:
# get rid of the first word of countries name
raw_data_fbref['Country'] = raw_data_fbref['Country'].str.split().str[1]
count = raw_data_fbref["Country"].value_counts()
count

Country
ENG    2372
FRA     169
GER     143
ESP     121
NED     110
ITA     104
BEL      78
POR      61
SCO      37
BRA      34
ARG      29
DEN      24
UKR      14
AUT      13
SWE      12
CZE      11
USA      10
SUI       9
NOR       7
COL       7
JPN       6
TUR       6
URU       5
POL       3
ECU       3
MEX       3
CRO       2
RUS       1
SRB       1
KSA       1
GRE       1
KOR       1
RSA       1
Name: count, dtype: int64

- Data of players currently playing in premier league may include their previous records from other leagues in which `Season` format is different. It is neccessary to adjust this for later stages.

In [19]:
season_count = raw_data_fbref['Season'].value_counts()
print(season_count)
print(f"total: {season_count.sum()}")

Season
2022-2023    468
2021-2022    454
2020-2021    442
2019-2020    430
2023-2024    429
2018-2019    359
2017-2018    317
2016-2017    242
2015-2016    126
2014-2015     41
2017          14
2018          14
2019          13
2021          12
2016          12
2022          10
2020           8
2015           4
2023           4
Name: count, dtype: int64
total: 3399


In [20]:
def format_season(season):
    if pd.notna(season) and '-' in season:
        return season
    elif pd.notna(season):
        return f"{season}-{int(season) + 1}"
    else:
        return season
raw_data_fbref['Season'] = raw_data_fbref['Season'].apply(format_season)

In [21]:
season_count = raw_data_fbref['Season'].value_counts()
print(season_count)
print(f"total: {season_count.sum()}")

Season
2022-2023    478
2021-2022    466
2020-2021    450
2019-2020    443
2023-2024    433
2018-2019    373
2017-2018    331
2016-2017    254
2015-2016    130
2014-2015     41
Name: count, dtype: int64
total: 3399


- Remove prefix number, and normalize league name in `Comp` column using `unicode` library.

In [22]:
raw_data_fbref['Comp'] = raw_data_fbref['Comp'].apply(unidecode)
raw_data_fbref['Comp'] = raw_data_fbref['Comp'].str.replace(r'^(\d+\.?\s*)+', '', regex=True)
comp_counts = raw_data_fbref['Comp'].value_counts()
comp_counts

Comp
Premier League        1583
Championship           421
Jr. PL2 -- Div. 1      190
Ligue 1                143
Bundesliga             140
Serie A                133
La Liga                112
Eredivisie              86
Jr. PL2 -- Div. 2       74
League One              72
Primeira Liga           61
First Division A        57
Premiership             33
Primera Div             29
Ligue 2                 26
League Two              25
Eerste Divisie          24
Superliga               24
Jr. PL2                 18
Pro League A            12
Jr. U19 Bundesliga      11
First League            11
Allsvenskan             10
Segunda Division         9
Super Lg                 9
MLS                      8
Serie B                  8
Primera A                7
National League          7
Super Lig                6
J1 League                5
Primera Division         5
Pro League               5
Eliteserien              4
First Division B         4
Liga                     3
Tippeligaen            

- Each player may hold multiple positions on the field based on the coach's tactics. However, for the sake of simplicity in this project, we will focus on their main position.

In [23]:
position_count = raw_data_fbref['Position'].value_counts()
position_count

Position
 FW-MF                313
 MF (CM-DM)           278
 DF (CB)              257
 FW-MF (AM)           232
 FW                   220
                     ... 
 DF (FB)                9
 DF-MF (DM-FB)          9
 MF (CM-DM, left)       7
 DF-MF (DM, right)      7
 FW-MF (WM, left)       6
Name: count, Length: 76, dtype: int64

We will take the first (main) position of each player.

In [24]:
raw_data_fbref['Position'] = raw_data_fbref['Position'].str.replace('-', ' ').str.split().str[0]
position_count = raw_data_fbref['Position'].value_counts()
position_count

Position
DF    1496
FW    1118
MF     785
Name: count, dtype: int64

**Handling missing values**

First we will have a look at the missing ratio of each columns:

In [25]:
missing_ratio = raw_data_fbref.isnull().mean() * 100
missing_ratio

Name              0.000000
Position          0.000000
PreferredFoot     0.000000
Season            0.000000
Age               0.000000
Squad             0.000000
Country           0.000000
Comp              0.000000
LgRank            0.000000
MP                0.000000
Starts            0.000000
Min               0.000000
90s               0.000000
Gls               0.000000
Ast               0.000000
G+A               0.000000
G-PK              0.000000
PK                0.000000
PKatt             0.000000
CrdY              0.000000
CrdR              0.000000
xG               29.008532
npxG             29.008532
xAG              29.008532
npxG+xAG         29.008532
PrgC              0.000000
PrgP              0.000000
PrgR              0.000000
Gls/90            0.000000
Ast/90            0.558988
G+A/90            0.558988
G-PK/90           0.000000
G+A-PK/90         0.558988
xG/90            29.008532
xAG/90           29.008532
xG+xAG/90        29.008532
npxG/90          29.008532
n

The missing values in the columns can be categorized into two groups. The first group consists of columns with a low missing ratio, even less than 1%. The second group includes columns with a substantial number of missing values, approximately around 30%.

- First step, we consider removing the rows that contains more than 50% NaN values.

In [26]:
raw_data_fbref = raw_data_fbref.dropna(thresh=(raw_data_fbref.shape[1] * 0.5))
raw_data_fbref.shape

(3399, 38)

- Despite certain columns having a high missing ratio, we opt to retain them for future analysis given their significance. As this pertains to football statistics, a more equitable approach involves filling the empty cells with mean values derived from players occupying similar positions on the field.

In [27]:
missing_cols = missing_ratio[missing_ratio > 0].index
raw_data_fbref[missing_cols] = raw_data_fbref.groupby('Position')[missing_cols].transform(lambda x: x.fillna(x.mean()))

In [28]:
missing_ratio = raw_data_fbref.isnull().mean() * 100
if (missing_ratio == 0).all():
    print("The data is now good!")
else:
    print("Still having missing values, please recheck!")

The data is now good!


**Processed data**

In [29]:
processed_data_fbref = raw_data_fbref
processed_data_fbref.head()

,Name,Position,PreferredFoot,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls/90,Ast/90,G+A/90,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90
0,William Saliba,DF,Right,2018-2019,17,Saint-Étienne,FRA,Ligue 1,4,16,13,1277,14.2,0,0,0,0,0,0,1,0,0.300000,0.300000,0.000000,0.300000,7,21,4,0.00,0.0,0.00,0.00,0.00,0.020000,0.000000,0.020000,0.020000,0.020000
1,William Saliba,DF,Right,2019-2020,18,Saint-Étienne,FRA,Ligue 1,17,12,11,992,11.0,0,0,0,0,0,0,0,0,0.200000,0.200000,0.100000,0.300000,20,33,7,0.00,0.0,0.00,0.00,0.00,0.020000,0.010000,0.030000,0.020000,0.030000
2,William Saliba,DF,Right,2020-2021,19,Nice,FRA,Ligue 1,9,20,20,1800,20.0,1,0,1,1,0,0,2,0,0.800000,0.800000,0.000000,0.900000,36,106,3,0.05,0.0,0.05,0.05,0.05,0.040000,0.000000,0.040000,0.040000,0.040000
3,William Saliba,DF,Right,2020-2021,19,Arsenal,ENG,Jr. PL2 -- Div. 1,10,6,6,526,5.8,0,0,0,0,0,0,1,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.00,0.0,0.00,0.00,0.00,0.058096,0.079837,0.137923,0.056746,0.136612
4,William Saliba,DF,Right,2021-2022,20,Marseille,FRA,Ligue 1,2,36,36,3240,36.0,0,0,0,0,0,0,5,0,1.700000,1.700000,0.400000,2.100000,50,220,14,0.00,0.0,0.00,0.00,0.00,0.050000,0.010000,0.060000,0.050000,0.060000


### **2.2. Transfermarkt data**

#### **Raw data inspect**

In [30]:
raw_data_trans = pd.read_csv("../../data/raw/raw_data_trans.csv")
raw_data_trans.head()

,Name,Season,Injury,from,until,Days,Games missed
0,Ederson,20/21,Virus,"Dec 27, 2020","Jan 12, 2021",16 days,3
1,Ederson,19/20,Ill,"Dec 30, 2019","Jan 7, 2020",8 days,3
2,Ederson,19/20,Ill,"Dec 8, 2019","Dec 14, 2019",6 days,1
3,Ederson,19/20,muscular problems,"Nov 6, 2019","Nov 22, 2019",16 days,4
4,Ederson,17/18,Facial injury,"Sep 10, 2017","Sep 12, 2017",2 days,-


It seems that, for this data table, the data is relatively clean, and there are no columns containing junk data. Therefore, removing columns is not yet necessary at the current time. However, some reformation must be done for synchronization with the previous data.

**Checking data shape**

In [31]:
trans_shape = raw_data_trans.shape
trans_shape

(3233, 7)

In [32]:
# testing if data include enough 1000 items
print(f"Current shape: {fbref_shape}")

if fbref_shape[0] > 1000:
    print(f"Data is good!")
else:
    print(f"Raw data is absolutely small. Please recheck!")

Current shape: (3944, 39)
Data is good!


**Checking duplicated rows**

In [33]:
index = raw_data_trans.index
detect_dup_series = index.duplicated(keep='first')
num_duplicated_rows = detect_dup_series.sum()

In [34]:
# checking containing duplicated rows
if num_duplicated_rows == 0:
    print(f"Raw data have no duplicated rows.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate the data.!")

Raw data have no duplicated rows.!


**Column informations**

In [35]:
raw_data_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3233 entries, 0 to 3232
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          3233 non-null   object
 1   Season        3233 non-null   object
 2   Injury        3233 non-null   object
 3   from          3233 non-null   object
 4   until         3233 non-null   object
 5   Days          3233 non-null   object
 6   Games missed  3233 non-null   object
dtypes: object(7)
memory usage: 176.9+ KB


The dataset appears to be well-maintained with no discernible issues, and there is no necessity to eliminate any columns or rows.

**Field meanings**

| Field           | Description                                                  |
|-----------------|--------------------------------------------------------------|
| **Name**        | The name of the player.                                      |
| **Season**      | The season during which the injury occurred.                 |
| **Injury**      | The type or nature of the injury sustained by the player.    |
| **From**        | The date when the player suffered the injury.                |
| **Until**       | The date when the player recovered from the injury.          |
| **Days**        | The duration of the injury in days.                          |
| **Games missed**| The number of games the player missed due to the injury.     |


**Reformating features and Adjusting data types**

- We need to change `Season` format to match the one of fbref data for later stage.

In [36]:
seas_count = raw_data_trans['Season'].value_counts()
print(seas_count)
print(f"Total: {seas_count.sum()}")

Season
20/21    598
21/22    508
19/20    474
18/19    460
22/23    414
17/18    201
23/24    165
16/17    156
15/16    116
14/15     85
13/14     56
Name: count, dtype: int64
Total: 3233


In [37]:
def format_trans_season (season):
    if pd.notna(season):
        year = season.split('/')
        return f"20{year[0]}-20{year[1]}"
    else:
        return season
raw_data_trans['Season'] = raw_data_trans['Season'].apply(format_trans_season)
seas_count = raw_data_trans['Season'].value_counts()
print(seas_count)
print(f"Total: {seas_count.sum()}")

Season
2020-2021    598
2021-2022    508
2019-2020    474
2018-2019    460
2022-2023    414
2017-2018    201
2023-2024    165
2016-2017    156
2015-2016    116
2014-2015     85
2013-2014     56
Name: count, dtype: int64
Total: 3233


- Adjusting `Days` format to int, and removing unnecessary 'days' string.

In [38]:
raw_data_trans  = raw_data_trans.copy()
raw_data_trans['Days'] = raw_data_trans['Days'].str.split().str[0].astype('int64')
raw_data_trans['Days'].dtypes

dtype('int64')

- We will adjust the date object into simpler format.

In [39]:
raw_data_trans['from'] = pd.to_datetime(raw_data_trans['from'], errors='coerce')
raw_data_trans['until'] = pd.to_datetime(raw_data_trans['until'], errors='coerce')
raw_data_trans[['from', 'until']].head()

,from,until
0,2020-12-27,2021-01-12
1,2019-12-30,2020-01-07
2,2019-12-08,2019-12-14
3,2019-11-06,2019-11-22
4,2017-09-10,2017-09-12


- The format of the `Games missed` column appears to be incorrect. Let's examine its values first.

In [40]:
missed_games_count = raw_data_trans['Games missed'].value_counts()
missed_games_count

Games missed
1      654
2      480
3      360
-      266
4      220
      ... 
104      1
62       1
54       1
82       1
91       1
Name: count, Length: 66, dtype: int64

It appears that the data uses some characters to represent zero games missed. To address this, we should replace these characters with 0 and then convert the column to the correct data type.

In [41]:
raw_data_trans['Games missed'] = pd.to_numeric(raw_data_trans['Games missed'].replace('[^\d]', '0', regex=True))
raw_data_trans['Games missed']

0        3
1        3
2        1
3        4
4        0
        ..
3228     1
3229     1
3230     8
3231     4
3232    16
Name: Games missed, Length: 3233, dtype: int64

**Handling missing values**

First we will have a look at the missing ratio of each columns:

In [42]:
missing_ratio = raw_data_trans.isnull().mean() * 100
missing_ratio

Name            0.000000
Season          0.000000
Injury          0.000000
from            0.000000
until           1.948655
Days            0.000000
Games missed    0.000000
dtype: float64

Only one column exhibits missing values, possibly as a result of the reformation process. Given the relatively small proportion and the potential hindrance in subsequent data processing, we have decided to remove the rows with missing values in this column.

In [43]:
raw_data_trans = raw_data_trans.dropna(subset=['until'])

In [44]:
missing_ratio = raw_data_trans.isnull().mean() * 100
if (missing_ratio == 0).all():
    print("The data is now good!")
else:
    print("Still having missing values, please recheck!")

The data is now good!


**Processed data**

In [45]:
processed_data_trans = raw_data_trans
processed_data_trans.head()

,Name,Season,Injury,from,until,Days,Games missed
0,Ederson,2020-2021,Virus,2020-12-27,2021-01-12,16,3
1,Ederson,2019-2020,Ill,2019-12-30,2020-01-07,8,3
2,Ederson,2019-2020,Ill,2019-12-08,2019-12-14,6,1
3,Ederson,2019-2020,muscular problems,2019-11-06,2019-11-22,16,4
4,Ederson,2017-2018,Facial injury,2017-09-10,2017-09-12,2,0


## 3. Merging data

**Merging 2 dataframes can help to attach additional informations for later stage**

There is a challenge where the names of players in two dataframes may not always match. Therefore, we require the assistance of a package called `fuzzywuzzy` to assess the similarity between two strings for merging purposes.

In [46]:
# create a dictionary for matching names of 2 Name columns of 2 dataframes
name_matches = {}
fbref_name = processed_data_fbref['Name'].unique()
trans_name = list(processed_data_trans['Name'].unique())

for name in fbref_name:
    if trans_name:
        best_match, score = process.extractOne(name, trans_name,scorer = fuzz.token_set_ratio)
        if score > 90:
            name_matches[name] = best_match
            trans_name.remove(best_match)
        else:
            name_matches[name] = name
    else:
        name_matches[name] = name


processed_data_fbref['Name'] = processed_data_fbref['Name'].map(name_matches)

# Merge based on both 'Name' and 'Season'
merged_df = pd.merge(processed_data_fbref, processed_data_trans , left_on=['Name', 'Season'], right_on=['Name', 'Season'], how='left')

In [47]:
# matching names found
pd.Series(name_matches)

William Saliba            William Saliba
Declan Rice                  Declan Rice
Bukayo Saka                  Bukayo Saka
Ben White                      Ben White
Martin Ødegaard          Martin Ødegaard
                             ...        
Jacob Bruun Larsen    Jacob Bruun Larsen
Nathan Redmond            Nathan Redmond
Jack Cork                      Jack Cork
Michael Obafemi          Michael Obafemi
Hjalmar Ekdal              Hjalmar Ekdal
Length: 389, dtype: object

In [48]:
merged_df

,Name,Position,PreferredFoot,Season,Age,Squad,Country,Comp,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,Gls/90,Ast/90,G+A/90,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,Injury,from,until,Days,Games missed
0,William Saliba,DF,Right,2018-2019,17,Saint-Étienne,FRA,Ligue 1,4,16,13,1277,14.2,0,0,0,0,0,0,1,0,0.300000,0.300000,0.000000,0.300000,7,21,4,0.00,0.00,0.00,0.00,0.00,0.020000,0.000000,0.020000,0.020000,0.020000,Hamstring injury,2019-05-23,2019-09-28,128.0,22.0
1,William Saliba,DF,Right,2019-2020,18,Saint-Étienne,FRA,Ligue 1,17,12,11,992,11.0,0,0,0,0,0,0,0,0,0.200000,0.200000,0.100000,0.300000,20,33,7,0.00,0.00,0.00,0.00,0.00,0.020000,0.010000,0.030000,0.020000,0.030000,Metatarsal fracture,2019-11-04,2020-01-20,77.0,19.0
2,William Saliba,DF,Right,2020-2021,19,Nice,FRA,Ligue 1,9,20,20,1800,20.0,1,0,1,1,0,0,2,0,0.800000,0.800000,0.000000,0.900000,36,106,3,0.05,0.00,0.05,0.05,0.05,0.040000,0.000000,0.040000,0.040000,0.040000,NaN,NaT,NaT,NaN,NaN
3,William Saliba,DF,Right,2020-2021,19,Arsenal,ENG,Jr. PL2 -- Div. 1,10,6,6,526,5.8,0,0,0,0,0,0,1,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.00,0.00,0.00,0.00,0.00,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaT,NaT,NaN,NaN
4,William Saliba,DF,Right,2021-2022,20,Marseille,FRA,Ligue 1,2,36,36,3240,36.0,0,0,0,0,0,0,5,0,1.700000,1.700000,0.400000,2.100000,50,220,14,0.00,0.00,0.00,0.00,0.00,0.050000,0.010000,0.060000,0.050000,0.060000,NaN,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566,Hjalmar Ekdal,DF,Right,2019-2020,20,Frej,SWE,Superettan,14,30,30,2700,30.0,2,0,2,2,0,0,2,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.07,0.00,0.07,0.07,0.07,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaT,NaT,NaN,NaN
4567,Hjalmar Ekdal,DF,Right,2020-2021,21,Sirius,SWE,Allsvenskan,10,26,22,1946,21.6,0,0,0,0,0,0,4,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.00,0.00,0.00,0.00,0.00,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaT,NaT,NaN,NaN
4568,Hjalmar Ekdal,DF,Right,2021-2022,22,Djurgården,SWE,Allsvenskan,3,26,25,2234,24.8,4,0,4,4,0,0,3,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.16,0.00,0.16,0.16,0.16,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaT,NaT,NaN,NaN
4569,Hjalmar Ekdal,DF,Right,2022-2023,23,Djurgården,SWE,Allsvenskan,2,24,23,2023,22.5,3,0,3,3,0,0,2,0,1.103445,1.072249,1.269282,2.340478,0,0,0,0.13,0.00,0.13,0.13,0.13,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaT,NaT,NaN,NaN


**Check if merging is correct**

In [49]:
assert len(processed_data_fbref['Name'].unique()) == len(name_matches.keys())
assert len(merged_df['Season'].unique()) == len(processed_data_fbref['Season'].unique())

## 4. Save processed data

In [50]:
# save data to processed data folder
merged_df.to_csv('../../data/processed/processed_fb_data.csv', index=False, encoding='utf-8')